In [13]:
import pandas as pd
from elftools.elf.elffile import ELFFile

In [14]:
config = "fdsc1_rasm"
old_csv = "~/faults_" + config + "_new.csv"
new_csv = "faults_" + config + "_names.csv"
bin_name = "../out_" + config + "_1.elf"

data = pd.read_csv(old_csv)
data

,attempt,stop_addr,stop_fn,delay,target,bitflip,code,seed
0,0,0x080093ea,prvProcessReceivedCommands_dup,0.719,0x20005ce7,32,0,3735928559
1,1,memset,"<main[xTaskStack]>,",0.033,0x20000276,32,0,3735928559
2,2,0x08004e22,vListInsert_dup,0.634,0x2000419f,64,0,3735928559
3,3,0x08005560,prvCopyDataFromQueue_dup,0.336,0x2000561c,2,0,3735928559
4,4,0x080001b6,memset,0.072,0x20003440,1,0,3735928559
...,...,...,...,...,...,...,...,...
59995,59995,0x080072c8,xTaskResumeAll_ret_dup,0.996,0x200015c4,32,0,3735928559
59996,59996,memcpy,"<ucStaticTimerQueueStorage_dup+96>,",0.339,0x200018aa,1,-1,3735928559
59997,59997,0x08011df0,Multiply_dup,0.483,0x200010d7,4,0,3735928559
59998,59998,0x08011ed2,Multiply_dup,0.918,0x200016e0,128,0,3735928559


In [15]:
symtable = dict()

with open(bin_name, 'rb') as elffile:
    for section in ELFFile(elffile).iter_sections():
        try:
            for sym in section.iter_symbols():
                if (sym.name != "ucHeap" and sym.entry['st_info']['type'] == 'STT_OBJECT'):
                    symtable[sym.name] = {"addr": sym.entry['st_value'], "size": sym.entry['st_size']}
        except: 
            continue

In [16]:
data_size = data.shape[0]

new_data = [None for i in range(data_size)]

def associate_symbols():

    for idx, row in data.iterrows():
        found = False
        for sym in symtable.keys():
            try:
                if int(row['target'][2:], base=16) in range(symtable[sym]['addr'], symtable[sym]['addr']+symtable[sym]['size'], 1):
                    #print(symtable[sym])
                    new_data[idx] = sym
                    break
            except:
                break
        print(str(idx) + "/" + str(data_size), end = "\r")

associate_symbols()

data['struct'] = new_data

In [17]:
data.to_csv(new_csv)

In [18]:
'''
    37: 20001900  2000 OBJECT  LOCAL  DEFAULT   10 main.xBufferRecvStack
    34: 20001068  2000 OBJECT  LOCAL  DEFAULT   10 main.xTaskStack
'''

print(hex(0x20001900 + 2000))
print(hex(0x20001068 + 2000))

0x200020d0
0x20001838
